In [ ]:
import requests
import civis
import json
import pandas as pd
import urllib.parse
import time
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
TABLES = ['Affiliates', 'Individual', 'State File']
TABLE_NAME = 'dept_secretarytreasurer.sys_officerbeneficiaries_monthlyexportlist_xn'

TODAY_DATE = (date.today()).strftime("%m/%d/%Y")
RECORD_UPDATE = 'No Action Needed, This is Correct.'
UPDATED_VAL = 'No'

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = 'e78349ac-c900-4089-83a0-66a86ae4e185'
        self.APP_ID = '6124ef7965dc077ddc4fc0ce'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower().replace(' ',''):item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})

    
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url + "&rows_per_page=1000"
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_record(self, knack_object, number):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['uid']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': number}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]
        else:
            return ''

    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.APP_DICT[connection_name]['obj_id']
        knack_field_id = self.APP_DICT[connection_name]['fields'][field]

        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        res_json_dict = json.loads(json.dumps(r.json()))

        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''

client = KnackAFT()

def run_payload(input_data):
    data_out = {}
    print(input_data['affiliateofficerid'])
    street = input_data.pop('addressline1')
    street2 = input_data.pop('addressline2')
    city = input_data.pop('city')
    state = input_data.pop('stateterritorycode')
    zipcode = input_data.pop('postalcode')
    
    firstname = input_data.pop('firstname')
    lastname = input_data.pop('lastname')
    
    currentemail = input_data.pop('bestemail')

    input_data.pop('affiliatename')
    input_data.update({
        'lastmaildate':TODAY_DATE,
        'lastemaildate':TODAY_DATE,
        'recordupdate':RECORD_UPDATE,
        'updated':UPDATED_VAL
    })

    input_data.update({'affiliatenumber':input_data['affiliatenumber'].lstrip('0')})
        
    input_data.update({'termstart':input_data['termstartdate']})
    input_data.pop('termstartdate')
    
    input_data.update({'termend':input_data['termenddate']})
    input_data.pop('termenddate')
    
    input_data.update({'currentofficerrole':input_data['officerroletypename']})
    input_data.pop('officerroletypename')
    
    input_data.update({'currentaffiliateofficerrolename':input_data['affiliateofficerrolename']})
    input_data.pop('affiliateofficerrolename')

    input_data.update({'locstabbr':input_data['locationstateabr']})
    input_data.pop('locationstateabr')
    
    aff_conn = client.get_connection('Affiliates', 'affiliatenumber', input_data['affiliatenumber'])
    input_data.update({'affiliatenumber':aff_conn})
    
    state_conn = client.get_connection('State File', 'statename', input_data['stateterritoryname'])
    input_data.update({'statefile':state_conn})
    input_data.pop('stateterritoryname')
    
    for k,v in input_data.items():
        data_out.update({client.APP_DICT['Individual']['fields'][k]:v})
    
    data_out.update({"field_3": {"email": currentemail}})
    data_out.update({"field_182": {
            "street": street,
            "street2": street2,
            "city": city,
            "state": state,
            "zip": zipcode
        }})
    data_out.update({"field_2": {"first": firstname,"last": lastname}})
    
    print(f"Creating new record for OfficerId: {input_data['affiliateofficerid']}...")
    request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["Individual"]['obj_id'] + "/records"
    r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(data_out))
    print(r)
    if r.status_code != 200 and r.status_code != 500:
        print(json.dumps(r.json(), indent=4))
    print('---------------------------------------')
    
    
def update_guids():
    print('Updating GUIDS...')
    records = client.find_matches('Individual', 'mdate', TODAY_DATE)
    for record in records:
        guid = record['id']
        
        print(f"Found record with AffiliateOfficerId: {record['field_64']}, updating GUID field to - {guid}")
        
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["Individual"]['obj_id'] + "/records/" + guid
        data_out = ({'field_143':guid})
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(data_out))
        print(r)
        print('---------------------------------------')

def upload_export_list():
    try:
        f = civis.io.read_civis(table=TABLE_NAME,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)
            
        df['termstartdate'] = pd.to_datetime(df.termstartdate)
        df['termstartdate'] = df['termstartdate'].dt.strftime('%m/%d/%Y')
            
        df['termenddate'] = pd.to_datetime(df.termenddate)
        df['termenddate'] = df['termenddate'].dt.strftime('%m/%d/%Y')

        for payload_dict in df.to_dict('records'):
            time.sleep(1)
            run_payload(payload_dict)

upload_export_list()
time.sleep(10)
update_guids()